In [1]:
import pandas as pd
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, Dense, GlobalMaxPooling1D

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('data/train.csv')
val_data = pd.read_csv('data/validation.csv')
print(len(train_data), len(val_data))

105000 15000


In [30]:
train_data.head()

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2


In [3]:
train_data.columns[2:]

Index(['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again'],
      dtype='object')

- 位置：3个，location_traffic_convenience, location_distance_from_business_district, location_easy_to_find
- 服务：4个，service_wait_time, service_waiters_attitude, service_parking_convenience, service_serving_speed
- 价格：3个，price_level, price_cost_effective, price_discount
- 环境：4个，environment_decoration, environment_noise, environment_space, environment_cleaness
- 菜品：4个，dish_portion, dish_taste, dish_look, dish_recommendation
- 其他：2个，others_overall_experience, others_willing_to_consume_again

In [4]:
x_train = list(map(lambda x: x[1:-1], train_data['content']))
x_val = list(map(lambda x: x[1:-1], val_data['content']))

In [7]:
tokenizer = Tokenizer(num_words=10000, char_level=True)
tokenizer.fit_on_texts(x_train)

In [8]:
sequences = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(sequences, maxlen=1000)

sequences = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(sequences, maxlen=1000)

In [5]:
y_train_ltc = pd.get_dummies(train_data['location_traffic_convenience'])[[-2, -1, 0, 1]].values
y_train_ldfbd = pd.get_dummies(train_data['location_distance_from_business_district'])[[-2, -1, 0, 1]].values
y_train_letf = pd.get_dummies(train_data['location_easy_to_find'])[[-2, -1, 0, 1]].values

y_train_swt = pd.get_dummies(train_data['service_wait_time'])[[-2, -1, 0, 1]].values
y_train_swa = pd.get_dummies(train_data['service_waiters_attitude'])[[-2, -1, 0, 1]].values
y_train_spc = pd.get_dummies(train_data['service_parking_convenience'])[[-2, -1, 0, 1]].values
y_train_sss = pd.get_dummies(train_data['service_serving_speed'])[[-2, -1, 0, 1]].values

y_train_pl = pd.get_dummies(train_data['price_level'])[[-2, -1, 0, 1]].values
y_train_pce = pd.get_dummies(train_data['price_cost_effective'])[[-2, -1, 0, 1]].values
y_train_pd = pd.get_dummies(train_data['price_discount'])[[-2, -1, 0, 1]].values

y_train_ed = pd.get_dummies(train_data['environment_decoration'])[[-2, -1, 0, 1]].values
y_train_en = pd.get_dummies(train_data['environment_noise'])[[-2, -1, 0, 1]].values
y_train_es = pd.get_dummies(train_data['environment_space'])[[-2, -1, 0, 1]].values
y_train_ec = pd.get_dummies(train_data['environment_cleaness'])[[-2, -1, 0, 1]].values

y_train_dp = pd.get_dummies(train_data['dish_portion'])[[-2, -1, 0, 1]].values
y_train_dt = pd.get_dummies(train_data['dish_taste'])[[-2, -1, 0, 1]].values
y_train_dl = pd.get_dummies(train_data['dish_look'])[[-2, -1, 0, 1]].values
y_train_dr = pd.get_dummies(train_data['dish_recommendation'])[[-2, -1, 0, 1]].values

y_train_ooe = pd.get_dummies(train_data['others_overall_experience'])[[-2, -1, 0, 1]].values
y_train_owtca = pd.get_dummies(train_data['others_willing_to_consume_again'])[[-2, -1, 0, 1]].values

In [6]:
y_val_ltc = pd.get_dummies(val_data['location_traffic_convenience'])[[-2, -1, 0, 1]].values
y_val_ldfbd = pd.get_dummies(val_data['location_distance_from_business_district'])[[-2, -1, 0, 1]].values
y_val_letf = pd.get_dummies(val_data['location_easy_to_find'])[[-2, -1, 0, 1]].values

y_val_swt = pd.get_dummies(val_data['service_wait_time'])[[-2, -1, 0, 1]].values
y_val_swa = pd.get_dummies(val_data['service_waiters_attitude'])[[-2, -1, 0, 1]].values
y_val_spc = pd.get_dummies(val_data['service_parking_convenience'])[[-2, -1, 0, 1]].values
y_val_sss = pd.get_dummies(val_data['service_serving_speed'])[[-2, -1, 0, 1]].values

y_val_pl = pd.get_dummies(val_data['price_level'])[[-2, -1, 0, 1]].values
y_val_pce = pd.get_dummies(val_data['price_cost_effective'])[[-2, -1, 0, 1]].values
y_val_pd = pd.get_dummies(val_data['price_discount'])[[-2, -1, 0, 1]].values

y_val_ed = pd.get_dummies(val_data['environment_decoration'])[[-2, -1, 0, 1]].values
y_val_en = pd.get_dummies(val_data['environment_noise'])[[-2, -1, 0, 1]].values
y_val_es = pd.get_dummies(val_data['environment_space'])[[-2, -1, 0, 1]].values
y_val_ec = pd.get_dummies(val_data['environment_cleaness'])[[-2, -1, 0, 1]].values

y_val_dp = pd.get_dummies(val_data['dish_portion'])[[-2, -1, 0, 1]].values
y_val_dt = pd.get_dummies(val_data['dish_taste'])[[-2, -1, 0, 1]].values
y_val_dl = pd.get_dummies(val_data['dish_look'])[[-2, -1, 0, 1]].values
y_val_dr = pd.get_dummies(val_data['dish_recommendation'])[[-2, -1, 0, 1]].values

y_val_ooe = pd.get_dummies(val_data['others_overall_experience'])[[-2, -1, 0, 1]].values
y_val_owtca = pd.get_dummies(val_data['others_willing_to_consume_again'])[[-2, -1, 0, 1]].values

In [9]:
def build_model():
    model = Sequential()
    model.add(Embedding(10000, 128, input_length=1000))
    model.add(Conv1D(32, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
    return model

#### 位置

In [10]:
model1 = build_model()
model1.fit(x_train, y_train_ltc, epochs=5, batch_size=64, validation_data=(x_val, y_val_ltc))

Train on 105000 samples, validate on 15000 samples
Epoch 1/5
105000/105000 [==============================] - 29s 272us/step - loss: 0.2570 - acc: 0.9269 - val_loss: 0.2043 - val_acc: 0.9404
Epoch 2/5
105000/105000 [==============================] - 25s 240us/step - loss: 0.2057 - acc: 0.9402 - val_loss: 0.1976 - val_acc: 0.9427
Epoch 3/5
105000/105000 [==============================] - 25s 241us/step - loss: 0.1948 - acc: 0.9430 - val_loss: 0.1988 - val_acc: 0.9427
Epoch 4/5
105000/105000 [==============================] - 25s 241us/step - loss: 0.1874 - acc: 0.9438 - val_loss: 0.2125 - val_acc: 0.9410
Epoch 5/5
105000/105000 [==============================] - 25s 241us/step - loss: 0.1831 - acc: 0.9456 - val_loss: 0.2124 - val_acc: 0.9399


In [11]:
model1 = build_model()
model1.fit(x_train, y_train_ldfbd, epochs=3, batch_size=64, validation_data=(x_val, y_val_ldfbd))

Train on 105000 samples, validate on 15000 samples
Epoch 1/3
105000/105000 [==============================] - 26s 245us/step - loss: 0.3522 - acc: 0.8754 - val_loss: 0.3275 - val_acc: 0.8825
Epoch 2/3
105000/105000 [==============================] - 25s 240us/step - loss: 0.3111 - acc: 0.8869 - val_loss: 0.3227 - val_acc: 0.8830
Epoch 3/3
105000/105000 [==============================] - 25s 240us/step - loss: 0.2985 - acc: 0.8903 - val_loss: 0.3236 - val_acc: 0.8821


In [12]:
model1 = build_model()
model1.fit(x_train, y_train_letf, epochs=2, batch_size=64, validation_data=(x_val, y_val_letf))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.3761 - acc: 0.8861 - val_loss: 0.3287 - val_acc: 0.9005
Epoch 2/2
105000/105000 [==============================] - 25s 241us/step - loss: 0.3017 - acc: 0.9062 - val_loss: 0.2994 - val_acc: 0.9097


#### 服务

In [13]:
model1 = build_model()
model1.fit(x_train, y_train_swt, epochs=2, batch_size=64, validation_data=(x_val, y_val_swt))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 247us/step - loss: 0.3330 - acc: 0.8932 - val_loss: 0.3033 - val_acc: 0.9031
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.2895 - acc: 0.9044 - val_loss: 0.2962 - val_acc: 0.9037


In [14]:
model1 = build_model()
model1.fit(x_train, y_train_swa, epochs=2, batch_size=64, validation_data=(x_val, y_val_swa))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 245us/step - loss: 0.5685 - acc: 0.7879 - val_loss: 0.4806 - val_acc: 0.8171
Epoch 2/2
105000/105000 [==============================] - 25s 240us/step - loss: 0.4511 - acc: 0.8327 - val_loss: 0.4569 - val_acc: 0.8270


In [15]:
model1 = build_model()
model1.fit(x_train, y_train_spc, epochs=2, batch_size=64, validation_data=(x_val, y_val_spc))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 246us/step - loss: 0.1157 - acc: 0.9647 - val_loss: 0.0923 - val_acc: 0.9696
Epoch 2/2
105000/105000 [==============================] - 25s 240us/step - loss: 0.0856 - acc: 0.9730 - val_loss: 0.0834 - val_acc: 0.9725


In [16]:
model1 = build_model()
model1.fit(x_train, y_train_sss, epochs=2, batch_size=64, validation_data=(x_val, y_val_sss))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 247us/step - loss: 0.3026 - acc: 0.9091 - val_loss: 0.2534 - val_acc: 0.9223
Epoch 2/2
105000/105000 [==============================] - 25s 241us/step - loss: 0.2375 - acc: 0.9257 - val_loss: 0.2563 - val_acc: 0.9226


#### 价格

In [17]:
model1 = build_model()
model1.fit(x_train, y_train_pl, epochs=2, batch_size=64, validation_data=(x_val, y_val_pl))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.6910 - acc: 0.7253 - val_loss: 0.5782 - val_acc: 0.7774
Epoch 2/2
105000/105000 [==============================] - 25s 243us/step - loss: 0.5523 - acc: 0.7868 - val_loss: 0.5835 - val_acc: 0.7706


In [18]:
model1 = build_model()
model1.fit(x_train, y_train_pce, epochs=2, batch_size=64, validation_data=(x_val, y_val_pce))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 247us/step - loss: 0.3989 - acc: 0.8643 - val_loss: 0.3445 - val_acc: 0.8805
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.3352 - acc: 0.8819 - val_loss: 0.3357 - val_acc: 0.8843


In [19]:
model1 = build_model()
model1.fit(x_train, y_train_pd, epochs=2, batch_size=64, validation_data=(x_val, y_val_pd))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 247us/step - loss: 0.5115 - acc: 0.8060 - val_loss: 0.4607 - val_acc: 0.8274
Epoch 2/2
105000/105000 [==============================] - 25s 241us/step - loss: 0.4430 - acc: 0.8327 - val_loss: 0.4457 - val_acc: 0.8317


#### 环境

In [20]:
model1 = build_model()
model1.fit(x_train, y_train_ed, epochs=2, batch_size=64, validation_data=(x_val, y_val_ed))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 248us/step - loss: 0.5559 - acc: 0.8045 - val_loss: 0.5459 - val_acc: 0.8075
Epoch 2/2
105000/105000 [==============================] - 26s 243us/step - loss: 0.4558 - acc: 0.8402 - val_loss: 0.4587 - val_acc: 0.8402


In [21]:
model1 = build_model()
model1.fit(x_train, y_train_en, epochs=2, batch_size=64, validation_data=(x_val, y_val_en))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 248us/step - loss: 0.4836 - acc: 0.8307 - val_loss: 0.4259 - val_acc: 0.8479
Epoch 2/2
105000/105000 [==============================] - 25s 242us/step - loss: 0.4026 - acc: 0.8593 - val_loss: 0.4080 - val_acc: 0.8541


In [22]:
model1 = build_model()
model1.fit(x_train, y_train_es, epochs=2, batch_size=64, validation_data=(x_val, y_val_es))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.6692 - acc: 0.7576 - val_loss: 0.5802 - val_acc: 0.7924
Epoch 2/2
105000/105000 [==============================] - 26s 244us/step - loss: 0.5216 - acc: 0.8118 - val_loss: 0.5230 - val_acc: 0.8078


In [23]:
model1 = build_model()
model1.fit(x_train, y_train_ec, epochs=2, batch_size=64, validation_data=(x_val, y_val_ec))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 250us/step - loss: 0.5315 - acc: 0.8190 - val_loss: 0.4541 - val_acc: 0.8495
Epoch 2/2
105000/105000 [==============================] - 26s 243us/step - loss: 0.4350 - acc: 0.8513 - val_loss: 0.4213 - val_acc: 0.8564


#### 菜品

In [24]:
model1 = build_model()
model1.fit(x_train, y_train_dp, epochs=2, batch_size=64, validation_data=(x_val, y_val_dp))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 259us/step - loss: 0.7264 - acc: 0.7342 - val_loss: 0.6801 - val_acc: 0.7515
Epoch 2/2
105000/105000 [==============================] - 26s 249us/step - loss: 0.6151 - acc: 0.7728 - val_loss: 0.6534 - val_acc: 0.7567


In [25]:
model1 = build_model()
model1.fit(x_train, y_train_dt, epochs=2, batch_size=64, validation_data=(x_val, y_val_dt))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 26s 250us/step - loss: 0.6795 - acc: 0.7079 - val_loss: 0.6268 - val_acc: 0.7302
Epoch 2/2
105000/105000 [==============================] - 26s 244us/step - loss: 0.5651 - acc: 0.7588 - val_loss: 0.5957 - val_acc: 0.7424


In [26]:
model1 = build_model()
model1.fit(x_train, y_train_dl, epochs=2, batch_size=64, validation_data=(x_val, y_val_dl))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 254us/step - loss: 0.6574 - acc: 0.7733 - val_loss: 0.6069 - val_acc: 0.7865
Epoch 2/2
105000/105000 [==============================] - 26s 245us/step - loss: 0.5820 - acc: 0.7936 - val_loss: 0.6024 - val_acc: 0.7889


In [27]:
model1 = build_model()
model1.fit(x_train, y_train_dr, epochs=2, batch_size=64, validation_data=(x_val, y_val_dr))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 27s 260us/step - loss: 0.3902 - acc: 0.8730 - val_loss: 0.3416 - val_acc: 0.8927
Epoch 2/2
105000/105000 [==============================] - 26s 251us/step - loss: 0.3257 - acc: 0.8940 - val_loss: 0.3315 - val_acc: 0.8937


#### 其他

In [28]:
model1 = build_model()
model1.fit(x_train, y_train_ooe, epochs=2, batch_size=64, validation_data=(x_val, y_val_ooe))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 28s 266us/step - loss: 0.6196 - acc: 0.7565 - val_loss: 0.5899 - val_acc: 0.7641
Epoch 2/2
105000/105000 [==============================] - 27s 253us/step - loss: 0.5365 - acc: 0.7923 - val_loss: 0.6029 - val_acc: 0.7614


In [29]:
model1 = build_model()
model1.fit(x_train, y_train_owtca, epochs=2, batch_size=64, validation_data=(x_val, y_val_owtca))

Train on 105000 samples, validate on 15000 samples
Epoch 1/2
105000/105000 [==============================] - 28s 264us/step - loss: 0.5464 - acc: 0.8011 - val_loss: 0.4762 - val_acc: 0.8249
Epoch 2/2
105000/105000 [==============================] - 27s 253us/step - loss: 0.4524 - acc: 0.8340 - val_loss: 0.4628 - val_acc: 0.8290
